In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os
import mgrs
from datetime import datetime, timedelta

# Step 1 : Load train/val/test breeding sets

In [2]:
df_train_paths = pd.read_csv('../kaggle_data/train_ds.csv')
#df_val_paths = pd.read_csv('hls_data_sample/val.csv')
df_test_paths = pd.read_csv('../kaggle_data/test_ds.csv')

df_train_labels = pd.read_csv('hls_data_sample/records/train.csv')
df_val_labels = pd.read_csv('hls_data_sample/records/val.csv')
df_test_labels = pd.read_csv('hls_data_sample/records/test.csv')

In [3]:
df_train_paths.head()

,Input,Label
0,hls_train/hls_train/chips/chip_20191201_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20191201_...
1,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...
2,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...
3,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...
4,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...


In [4]:
df_train_labels

,Unnamed: 0,x,y,date,label,generated,CAT,year
0,6507,38.688200,8.409860,2020-04-17,0.0,1,PseudoAbsence,2020
1,22,28.588200,11.109860,2020-06-23,0.0,1,PseudoAbsence,2020
2,1071,40.688200,30.509860,2020-10-08,0.0,1,PseudoAbsence,2020
3,4038,8.588200,20.409860,2021-04-16,0.0,1,PseudoAbsence,2021
4,5759,46.788200,-18.690140,2020-08-31,0.0,1,PseudoAbsence,2020
5,2503,1.488200,8.109860,2020-02-19,0.0,1,PseudoAbsence,2020
6,4664,72.688200,22.309860,2020-06-11,0.0,1,PseudoAbsence,2020
7,241,54.588200,24.509860,2020-01-31,0.0,1,PseudoAbsence,2020
8,588,0.088200,20.509860,2020-07-18,0.0,1,PseudoAbsence,2020
9,7519,24.288200,21.809860,2020-10-04,0.0,1,PseudoAbsence,2020


In [5]:
def convert_timestamp(timestamp):
    # Extract year, day of year, and time components from the timestamp
    year = int(timestamp[:4])
    day_of_year = int(timestamp[4:7])
    time_str = timestamp[8:]
    
    # Create a datetime object for January 1st of the given year
    jan_first = datetime(year, 1, 1)
    
    # Calculate the exact date by adding the day of year offset
    date = jan_first + timedelta(days=day_of_year - 1)
    
    # Parse the time component
    time = datetime.strptime(time_str, "%H%M%S").time()
    
    # Combine date and time into a single datetime object
    combined_datetime = datetime.combine(date, time)
    
    return combined_datetime

In [6]:
def retrieve_info_filename(df):
    
    for col in ['record_date', 'tile_id', 
                #'record_lat', 'record_lon', 
                'hls_date', 'hls_hour']:
        if col not in df.columns:
            df[col] = np.nan
    
#    df_train_paths[['record_date', 'record_lat', 'record_lon', 'hls_date', 'hls_hour']] = np.nan

    for index,row in df.iterrows():
        raw_path = row['Input'].split('chip_')[-1].split('.tif')[0]
        
        record_date = raw_path.split('_')[0]
        record_date = record_date[0:4] + '-' + record_date[4:6] + '-' + record_date[6:8]

        mgrs_code = raw_path.split('_')[2][1:]
        record_lat, record_lon = mgrs.MGRS().toLatLon(mgrs_code)
        
        record_timestamp = raw_path.split('_')[3]
        record_fulldate = convert_timestamp(record_timestamp)
        hls_date, hls_hour = record_fulldate.date(), record_fulldate.hour

        df.loc[index,'record_date'] = record_date
        df.loc[index,'tile_id'] = mgrs_code
        #df.loc[index,'record_lat'] = record_lat
        #df.loc[index,'record_lon'] = record_lon
        df.loc[index,'hls_date'] = hls_date
        df.loc[index,'hls_hour'] = hls_hour

    return df

In [7]:
df_train_paths = retrieve_info_filename(df_train_paths)
#df_val_paths = retrieve_info_filename(df_val_paths)
df_test_paths = retrieve_info_filename(df_test_paths)

/var/folders/nd/6d7z70s12vxdq276hh5_45z80000gn/T/ipykernel_72856/3388998116.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2019-12-01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index,'record_date'] = record_date
/var/folders/nd/6d7z70s12vxdq276hh5_45z80000gn/T/ipykernel_72856/3388998116.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39PXT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index,'tile_id'] = mgrs_code
/var/folders/nd/6d7z70s12vxdq276hh5_45z80000gn/T/ipykernel_72856/3388998116.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2019-10-30' has dtype incompatible with float64, please explicitly cast to a compatib

In [8]:
df_train_paths

,Input,Label,record_date,tile_id,hls_date,hls_hour
0,hls_train/hls_train/chips/chip_20191201_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20191201_...,2019-12-01,39PXT,2019-10-30,6.0
1,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,37QGA,2017-10-01,7.0
2,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...,2017-02-01,42RXR,2017-01-03,5.0
3,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...,2016-11-01,43RCL,2016-10-01,5.0
4,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...,2018-03-01,42RYQ,2018-01-31,5.0
...,...,...,...,...,...,...
10423,hls_train/hls_train/chips/chip_20201101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20201101_...,2020-11-01,38RQV,2020-10-02,7.0
10424,hls_train/hls_train/chips/chip_20200501_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200501_...,2020-05-01,40RFP,2020-03-31,6.0
10425,hls_train/hls_train/chips/chip_20200801_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200801_...,2020-08-01,43RBK,2020-07-01,5.0
10426,hls_train/hls_train/chips/chip_20170801_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20170801_...,2017-08-01,39QTU,2017-07-02,7.0


In [9]:
tiles_info = pd.read_csv('../kaggle_data/tiles_info.csv')
tiles_queries = pd.read_csv('../kaggle_data/tiles_queries.csv')
tiles_queries

,tile_id,Date1,Date2,Date3
0,37QET,2016-10-02,2016-09-22,2016-09-12
1,37NCD,2020-05-02,2020-04-22,2020-04-12
2,42RYR,2019-07-02,2019-06-22,2019-06-12
3,37PET,2020-09-01,2020-08-22,2020-08-12
4,38QKD,2019-10-02,2019-09-22,2019-09-12
...,...,...,...,...
22653,36QYL,2017-11-01,2017-10-22,2017-10-12
22654,37QDV,2017-01-30,2017-01-20,2017-01-10
22655,38QJE,2019-11-01,2019-10-22,2019-10-12
22656,43RDM,2020-07-02,2020-06-22,2020-06-12


In [10]:
tiles_info

,Unnamed: 0,tile_id,min_date,max_date,lon_min,lon_max,lat_min,lat_max
0,0,28QDF,2016-07-08,2020-11-06,-15.695278,-15.003333,18.095278,18.983889
1,1,28QEG,2016-08-07,2020-11-06,-14.998333,-14.049444,19.016111,19.891667
2,2,28QFG,2015-10-07,2020-08-07,-14.038611,-13.103889,18.990833,19.867500
3,3,28QFH,2015-11-07,2020-08-07,-13.971667,-13.083611,19.896667,20.787778
4,4,28QGL,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944
...,...,...,...,...,...,...,...,...
84,84,43RCM,2016-02-06,2020-07-07,72.959722,73.975278,28.021667,28.921389
85,85,43RDK,2015-11-07,2020-07-07,73.994167,74.953889,26.311667,27.117500
86,86,43RDL,2015-11-07,2020-11-06,73.988611,74.997778,27.123611,28.024722
87,87,43RDM,2015-11-07,2020-10-07,73.976667,74.997778,28.030000,28.923611


In [11]:
df_train_paths

,Input,Label,record_date,tile_id,hls_date,hls_hour
0,hls_train/hls_train/chips/chip_20191201_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20191201_...,2019-12-01,39PXT,2019-10-30,6.0
1,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,37QGA,2017-10-01,7.0
2,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...,2017-02-01,42RXR,2017-01-03,5.0
3,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...,2016-11-01,43RCL,2016-10-01,5.0
4,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...,2018-03-01,42RYQ,2018-01-31,5.0
...,...,...,...,...,...,...
10423,hls_train/hls_train/chips/chip_20201101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20201101_...,2020-11-01,38RQV,2020-10-02,7.0
10424,hls_train/hls_train/chips/chip_20200501_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200501_...,2020-05-01,40RFP,2020-03-31,6.0
10425,hls_train/hls_train/chips/chip_20200801_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200801_...,2020-08-01,43RBK,2020-07-01,5.0
10426,hls_train/hls_train/chips/chip_20170801_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20170801_...,2017-08-01,39QTU,2017-07-02,7.0


In [12]:
df_train_enriched = df_train_paths.merge(
    tiles_info,
    left_on=['tile_id'],
    right_on=['tile_id'],
    how='inner'
)

df_train_enriched

,Input,Label,record_date,tile_id,hls_date,hls_hour,Unnamed: 0,min_date,max_date,lon_min,lon_max,lat_min,lat_max
0,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,37QGA,2017-10-01,7.0,26,2015-11-07,2020-11-06,41.193611,41.763611,18.079444,18.975278
1,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...,2017-02-01,42RXR,2017-01-03,5.0,73,2015-12-08,2020-07-07,70.016944,71.025833,27.120000,28.017778
2,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...,2016-11-01,43RCL,2016-10-01,5.0,83,2015-11-07,2020-07-07,72.968889,73.989167,27.117500,28.000278
3,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...,2018-03-01,42RYQ,2018-01-31,5.0,76,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778
4,hls_train/hls_train/chips/chip_20191101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20191101_...,2019-11-01,36QZK,2019-09-30,8.0,9,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5900,hls_train/hls_train/chips/chip_20171101_L30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,38QKE,2017-10-02,7.0,37,2016-06-07,2020-09-06,42.171944,43.095278,17.173611,18.057778
5901,hls_train/hls_train/chips/chip_20200901_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200901_...,2020-09-01,42RTP,2020-08-04,6.0,64,2016-03-07,2020-10-07,66.016111,66.964722,25.301944,26.176389
5902,hls_train/hls_train/chips/chip_20200601_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20200601_...,2020-06-01,36NZH,2020-05-01,7.0,6,2020-03-07,2020-05-07,35.699722,35.858333,2.158056,2.538056
5903,hls_train/hls_train/chips/chip_20200501_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200501_...,2020-05-01,40RFP,2020-03-31,6.0,49,2016-01-06,2020-11-06,58.003333,58.917222,25.612778,25.949722


In [13]:
# df_train_enriched.to_csv('train_enriched.csv',index=False)

# Step 2 : Load MERRA-2 dataset at same date

In [14]:
import earthaccess
import math

In [15]:
auth = earthaccess.login(strategy="interactive", persist=True)

In [16]:
df_train_enriched.head()

,Input,Label,record_date,tile_id,hls_date,hls_hour,Unnamed: 0,min_date,max_date,lon_min,lon_max,lat_min,lat_max
0,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,37QGA,2017-10-01,7.0,26,2015-11-07,2020-11-06,41.193611,41.763611,18.079444,18.975278
1,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...,2017-02-01,42RXR,2017-01-03,5.0,73,2015-12-08,2020-07-07,70.016944,71.025833,27.120000,28.017778
2,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...,2016-11-01,43RCL,2016-10-01,5.0,83,2015-11-07,2020-07-07,72.968889,73.989167,27.117500,28.000278
3,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...,2018-03-01,42RYQ,2018-01-31,5.0,76,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778
4,hls_train/hls_train/chips/chip_20191101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20191101_...,2019-11-01,36QZK,2019-09-30,8.0,9,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444


In [17]:
dedup_df_train_enriched = df_train_enriched[[
    'tile_id','hls_date','hls_hour', 'record_date',
    'min_date',	'max_date',	'lon_min',	'lon_max',	'lat_min',	'lat_max'
]].drop_duplicates().sort_values('hls_date',ascending=False)

# dedup_df_train_enriched.to_csv('train_enriched_dedup.csv',index=False)

dedup_df_train_enriched

,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max
1479,36QZK,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444
1562,36QYL,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,34.978056,35.905278,22.581111,23.489444
472,28QDF,2020-11-03,11.0,2020-12-01,2016-07-08,2020-11-06,-15.695278,-15.003333,18.095278,18.983889
1544,37QFC,2020-11-02,7.0,2020-12-01,2016-06-07,2020-11-06,39.961944,40.910278,19.887222,20.796389
720,42RXP,2020-11-01,5.0,2020-12-01,2017-02-05,2020-11-06,69.997500,70.992222,25.308333,26.213611
...,...,...,...,...,...,...,...,...,...,...
727,43RBL,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,72.000000,72.976667,27.103056,28.005833
1572,42RYQ,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778
3863,40REP,2015-12-01,6.0,2016-01-01,2015-11-07,2020-11-06,57.239722,57.977222,25.643889,26.208056
2239,37QDU,2015-11-29,7.0,2016-01-01,2015-11-07,2020-11-06,38.077778,38.998889,16.285278,17.173333


In [18]:
merra_generated_files = ['_'.join(x[:-4].split('_')[1:]) for x in os.listdir('merra_data')]

dedup_df_train_enriched_already_generated = dedup_df_train_enriched.loc[
    dedup_df_train_enriched.apply(
        lambda x : (str(x['hls_date'])+'_'+x['tile_id']+'_'+str(x['hls_hour'])) in (merra_generated_files),
        axis=1
    )
]

dedup_df_train_enriched_already_generated

,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max
720,42RXP,2020-11-01,5.0,2020-12-01,2017-02-05,2020-11-06,69.997500,70.992222,25.308333,26.213611
728,39NUJ,2020-11-01,6.0,2020-12-01,2020-02-06,2020-11-06,49.287222,49.778611,7.445556,7.917778
15,38PMV,2020-11-01,7.0,2020-12-01,2016-03-07,2020-11-06,44.199444,44.992222,12.670833,13.423889
155,40RFP,2020-11-01,6.0,2020-12-01,2016-01-06,2020-11-06,58.003333,58.917222,25.612778,25.949722
114,40RFR,2020-11-01,6.0,2020-12-01,2016-02-06,2020-11-06,58.016667,59.026111,27.116389,27.966667
...,...,...,...,...,...,...,...,...,...,...
513,28QGL,2016-03-01,11.0,2016-04-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944
36,40RGP,2016-02-03,6.0,2016-03-01,2016-01-06,2020-11-06,59.215000,59.983889,25.379444,26.112778
416,36QWL,2016-02-03,8.0,2016-03-01,2015-11-07,2019-08-07,33.064167,33.488889,22.630000,23.440000
581,28QGL,2016-01-29,11.0,2016-03-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944


In [19]:
dedup_df_train_enriched_remaining = pd.concat([
    dedup_df_train_enriched,dedup_df_train_enriched_already_generated
]).drop_duplicates(keep=False).sort_values('hls_date',ascending=False)

dedup_df_train_enriched_remaining

,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max
1479,36QZK,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444
472,28QDF,2020-11-03,11.0,2020-12-01,2016-07-08,2020-11-06,-15.695278,-15.003333,18.095278,18.983889
1562,36QYL,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,34.978056,35.905278,22.581111,23.489444
1544,37QFC,2020-11-02,7.0,2020-12-01,2016-06-07,2020-11-06,39.961944,40.910278,19.887222,20.796389
1066,38PLS,2020-11-01,7.0,2020-12-01,2015-11-07,2020-11-06,43.176389,44.083056,9.946667,10.838333
...,...,...,...,...,...,...,...,...,...,...
727,43RBL,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,72.000000,72.976667,27.103056,28.005833
1572,42RYQ,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778
3863,40REP,2015-12-01,6.0,2016-01-01,2015-11-07,2020-11-06,57.239722,57.977222,25.643889,26.208056
2239,37QDU,2015-11-29,7.0,2016-01-01,2015-11-07,2020-11-06,38.077778,38.998889,16.285278,17.173333


In [ ]:
dedup_df_train_enriched_remaining = pd.concat([
    dedup_df_train_enriched,dedup_df_train_enriched_already_generated
]).drop_duplicates(keep=False).sort_values('hls_date',ascending=False)

dedup_df_train_enriched_remaining

In [28]:
overlap_df_train_enriched_remaining = pd.merge(
    dedup_df_train_enriched,dedup_df_train_enriched_already_generated, 
    how='inner', 
    on=['tile_id','hls_date','hls_hour','record_date','min_date','max_date','lon_min','lon_max','lat_min','lat_max']
)
overlap_df_train_enriched_remaining

,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max
0,42RXP,2020-11-01,5.0,2020-12-01,2017-02-05,2020-11-06,69.997500,70.992222,25.308333,26.213611
1,39NUJ,2020-11-01,6.0,2020-12-01,2020-02-06,2020-11-06,49.287222,49.778611,7.445556,7.917778
2,38PMV,2020-11-01,7.0,2020-12-01,2016-03-07,2020-11-06,44.199444,44.992222,12.670833,13.423889
3,40RFP,2020-11-01,6.0,2020-12-01,2016-01-06,2020-11-06,58.003333,58.917222,25.612778,25.949722
4,40RFR,2020-11-01,6.0,2020-12-01,2016-02-06,2020-11-06,58.016667,59.026111,27.116389,27.966667
...,...,...,...,...,...,...,...,...,...,...
311,28QGL,2016-03-01,11.0,2016-04-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944
312,40RGP,2016-02-03,6.0,2016-03-01,2016-01-06,2020-11-06,59.215000,59.983889,25.379444,26.112778
313,36QWL,2016-02-03,8.0,2016-03-01,2015-11-07,2019-08-07,33.064167,33.488889,22.630000,23.440000
314,28QGL,2016-01-29,11.0,2016-03-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944


In [26]:
import math
import os
import concurrent.futures
import pandas as pd
import xarray as xr
from datetime import timedelta
from collections import defaultdict
import earthaccess

def download_date(date):
    """Download data for a specific date and return the file path."""
    date_str = date.strftime('%Y-%m-%d')
    try:
        results = earthaccess.search_data(
            doi="10.5067/7MCPBJ41Y0K6",
            temporal=(date_str, date_str),
        )
        files = earthaccess.download(results, local_path='merra2_data')
        return (date, files[0]) if files else (date, None)
    except Exception as e:
        print(f"Download failed for {date_str}: {e}")
        return (date, None)

def get_merra_granule(df):
    auth = earthaccess.login()
    print(f"Working directory: {os.getcwd()}")

    # Precompute boundaries
    df['lat_min_floor'] = df['lat_min'].apply(math.floor)
    df['lat_max_ceil'] = df['lat_max'].apply(math.ceil)
    df['lon_min_floor'] = df['lon_min'].apply(math.floor)
    df['lon_max_ceil'] = df['lon_max'].apply(math.ceil)
    
    # Initialize output structure
    df['merra_path'] = df.apply(lambda _: ['', '', ''], axis=1)

    # Map all dates to their required rows and time slots
    date_map = defaultdict(list)
    required_dates = set()
    for index, row in df.iterrows():
        base_date = row['hls_date']
        for offset, slot in zip([0, -10, -20], [0, 1, 2]):
            target_date = base_date + timedelta(days=offset)
            required_dates.add(target_date)
            date_map[target_date].append((index, slot))

    # Process downloads concurrently
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # Submit all download tasks
        future_to_date = {
            executor.submit(download_date, date): date 
            for date in required_dates
        }

        # Process each download as it completes
        for future in concurrent.futures.as_completed(future_to_date):
            date = future_to_date[future]
            try:
                date_result, file_path = future.result()
                if not file_path:
                    continue

                # Process all rows needing this date
                with xr.open_dataset(file_path) as ds:
                    for index, slot in date_map.get(date_result, []):
                        row = df.loc[index]
                        
                        # Filter dataset
                        filtered = ds.sel(
                            lat=slice(row['lat_min_floor'], row['lat_max_ceil']),
                            lon=slice(row['lon_min_floor'], row['lon_max_ceil']),
                            time=ds.time.dt.hour == row['hls_hour']
                        )
                        
                        # Generate filename
                        date_str = date_result.strftime('%Y-%m-%d')
                        filename = f"merra2_data/merra2_{date_str}_{row['tile_id']}_{row['hls_hour']}.nc4"
                        filtered.to_netcdf(filename)
                        
                        # Update path list
                        paths = df.at[index, 'merra_path']
                        paths[slot] = filename
                        df.at[index, 'merra_path'] = paths

                # Cleanup original file immediately
                os.remove(file_path)
                print(f"Processed and cleaned up {date_result}")

            except Exception as e:
                print(f"Error processing {date}: {e}")

    # Convert path lists to strings
    df['merra_path'] = df['merra_path'].apply(lambda x: ' & '.join(x))
    return df

In [23]:
%%time
merra_enriched_train_dedup = get_merra_granule(dedup_df_train_enriched_remaining)

merra_enriched_train_dedup

Working directory: /Users/hakimdiouane/Documents/desert_locust_breeding_detection/benchmark_desert_locust_detection/Desert_Locust_Breeding_Detection/instageo_wxc/data_pipeline


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

...

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Processed and cleaned up 2019-11-01


COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Processed and cleaned up 2020-08-24
CPU times: user 10min 39s, sys: 17min 30s, total: 28min 9s
Wall time: 1h 28min 49s


,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max,lat_min_floor,lat_max_ceil,lon_min_floor,lon_max_ceil,merra_path
1479,36QZK,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444,22,23,35,36,merra2_data/merra2_2020-11-03_36QZK_8.0.nc4 & ...
472,28QDF,2020-11-03,11.0,2020-12-01,2016-07-08,2020-11-06,-15.695278,-15.003333,18.095278,18.983889,18,19,-16,-15,merra2_data/merra2_2020-11-03_28QDF_11.0.nc4 &...
1562,36QYL,2020-11-03,8.0,2020-12-01,2015-11-07,2020-11-06,34.978056,35.905278,22.581111,23.489444,22,24,34,36,merra2_data/merra2_2020-11-03_36QYL_8.0.nc4 & ...
1544,37QFC,2020-11-02,7.0,2020-12-01,2016-06-07,2020-11-06,39.961944,40.910278,19.887222,20.796389,19,21,39,41,merra2_data/merra2_2020-11-02_37QFC_7.0.nc4 & ...
1066,38PLS,2020-11-01,7.0,2020-12-01,2015-11-07,2020-11-06,43.176389,44.083056,9.946667,10.838333,9,11,43,45,merra2_data/merra2_2020-11-01_38PLS_7.0.nc4 & ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,43RBL,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,72.000000,72.976667,27.103056,28.005833,27,29,72,73,merra2_data/merra2_2015-12-02_43RBL_5.0.nc4 & ...
1572,42RYQ,2015-12-02,5.0,2016-01-01,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778,26,28,71,72,merra2_data/merra2_2015-12-02_42RYQ_5.0.nc4 & ...
3863,40REP,2015-12-01,6.0,2016-01-01,2015-11-07,2020-11-06,57.239722,57.977222,25.643889,26.208056,25,27,57,58,merra2_data/merra2_2015-12-01_40REP_6.0.nc4 & ...
2239,37QDU,2015-11-29,7.0,2016-01-01,2015-11-07,2020-11-06,38.077778,38.998889,16.285278,17.173333,16,18,38,39,merra2_data/merra2_2015-11-29_37QDU_7.0.nc4 & ...


In [29]:
%%time
merra_enriched_train_overlap = get_merra_granule(overlap_df_train_enriched_remaining)

merra_enriched_train_overlap

Working directory: /Users/hakimdiouane/Documents/desert_locust_breeding_detection/benchmark_desert_locust_detection/Desert_Locust_Breeding_Detection/instageo_wxc/data_pipeline


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

...

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Processed and cleaned up 2019-07-12


COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Processed and cleaned up 2016-10-31
CPU times: user 4min 21s, sys: 7min 29s, total: 11min 51s
Wall time: 36min 11s


,tile_id,hls_date,hls_hour,record_date,min_date,max_date,lon_min,lon_max,lat_min,lat_max,lat_min_floor,lat_max_ceil,lon_min_floor,lon_max_ceil,merra_path
0,42RXP,2020-11-01,5.0,2020-12-01,2017-02-05,2020-11-06,69.997500,70.992222,25.308333,26.213611,25,27,69,71,merra2_data/merra2_2020-11-01_42RXP_5.0.nc4 & ...
1,39NUJ,2020-11-01,6.0,2020-12-01,2020-02-06,2020-11-06,49.287222,49.778611,7.445556,7.917778,7,8,49,50,merra2_data/merra2_2020-11-01_39NUJ_6.0.nc4 & ...
2,38PMV,2020-11-01,7.0,2020-12-01,2016-03-07,2020-11-06,44.199444,44.992222,12.670833,13.423889,12,14,44,45,merra2_data/merra2_2020-11-01_38PMV_7.0.nc4 & ...
3,40RFP,2020-11-01,6.0,2020-12-01,2016-01-06,2020-11-06,58.003333,58.917222,25.612778,25.949722,25,26,58,59,merra2_data/merra2_2020-11-01_40RFP_6.0.nc4 & ...
4,40RFR,2020-11-01,6.0,2020-12-01,2016-02-06,2020-11-06,58.016667,59.026111,27.116389,27.966667,27,28,58,60,merra2_data/merra2_2020-11-01_40RFR_6.0.nc4 & ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,28QGL,2016-03-01,11.0,2016-04-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944,22,24,-13,-12,merra2_data/merra2_2016-03-01_28QGL_11.0.nc4 &...
312,40RGP,2016-02-03,6.0,2016-03-01,2016-01-06,2020-11-06,59.215000,59.983889,25.379444,26.112778,25,27,59,60,merra2_data/merra2_2016-02-03_40RGP_6.0.nc4 & ...
313,36QWL,2016-02-03,8.0,2016-03-01,2015-11-07,2019-08-07,33.064167,33.488889,22.630000,23.440000,22,24,33,34,merra2_data/merra2_2016-02-03_36QWL_8.0.nc4 & ...
314,28QGL,2016-01-29,11.0,2016-03-01,2015-10-07,2017-10-07,-12.876667,-12.066111,22.597222,23.451944,22,24,-13,-12,merra2_data/merra2_2016-01-29_28QGL_11.0.nc4 &...


In [49]:
# This will work if Earthdata prerequisite files have already been generated
auth = earthaccess.login()

# We recommend only streaming a few granules at a time, as some collections can be quite large
results = earthaccess.search_data(
    doi="10.5067/7MCPBJ41Y0K6",
    temporal=('2020-05-04','2020-05-04'), # This will stream one granule, but can be edited for a longer temporal extent
    #bounding_box=(47-1, 15-1, 47+1, 15+1)
)

epsilon = 1
lat_min, lat_max = 15-epsilon, 15+epsilon  # Latitude bounds
lon_min, lon_max = 47-epsilon, 47+epsilon  # Longitude bounds
hour_slice = 7

downloaded_files = earthaccess.download(
    results,
    local_path='merra_data', # Change this string to download to a different path
)

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
downloaded_files

['merra_data/MERRA2_400.tavg1_2d_flx_Nx.20200504.nc4']

In [71]:
ds = xr.open_dataset('merra_data/MERRA2_400.tavg1_2d_flx_Nx.20190930.nc4') # Opens the two granules and concatenates them along the time dimension
ds

<xarray.Dataset> Size: 918MB
Dimensions:      (lon: 576, lat: 361, time: 24)
Coordinates:
  * lon          (lon) float64 5kB -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * lat          (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time         (time) datetime64[ns] 192B 2019-09-30T00:30:00 ... 2019-09-3...
Data variables: (12/46)
    BSTAR        (time, lat, lon) float32 20MB ...
    CDH          (time, lat, lon) float32 20MB ...
    CDM          (time, lat, lon) float32 20MB ...
    CDQ          (time, lat, lon) float32 20MB ...
    CN           (time, lat, lon) float32 20MB ...
    DISPH        (time, lat, lon) float32 20MB ...
    ...           ...
    TSTAR        (time, lat, lon) float32 20MB ...
    ULML         (time, lat, lon) float32 20MB ...
    USTAR        (time, lat, lon) float32 20MB ...
    VLML         (time, lat, lon) float32 20MB ...
    Z0H          (time, lat, lon) float32 20MB ...
    Z0M          (time, lat, lon) float32 20MB ...
Attributes: (12/30)
    History:                           Original file generated: Fri Oct 11 01...
    Comment:                           GMAO filename: d5124_m2_jan10.tavg1_2d...
    Filename:                          MERRA2_400.tavg1_2d_flx_Nx.20190930.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/7MCPBJ41Y0K6
    RangeBeginningDate:                2019-09-30
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2019-09-30
    RangeEndingTime:                   23:59:59.000000

In [72]:
df_train_enriched.iloc[0:5]

,Input,Label,record_date,tile_id,hls_date,hls_hour,Unnamed: 0,min_date,max_date,lon_min,lon_max,lat_min,lat_max
0,hls_train/hls_train/chips/chip_20171101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20171101_...,2017-11-01,37QGA,2017-10-01,7.0,26,2015-11-07,2020-11-06,41.193611,41.763611,18.079444,18.975278
1,hls_train/hls_train/chips/chip_20170201_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20170201_...,2017-02-01,42RXR,2017-01-03,5.0,73,2015-12-08,2020-07-07,70.016944,71.025833,27.120000,28.017778
2,hls_train/hls_train/chips/chip_20161101_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20161101_...,2016-11-01,43RCL,2016-10-01,5.0,83,2015-11-07,2020-07-07,72.968889,73.989167,27.117500,28.000278
3,hls_train/hls_train/chips/chip_20180301_L30_T4...,hls_train/hls_train/seg_maps/seg_map_20180301_...,2018-03-01,42RYQ,2018-01-31,5.0,76,2015-11-07,2020-10-07,71.009722,71.998333,26.217500,27.102778
4,hls_train/hls_train/chips/chip_20191101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20191101_...,2019-11-01,36QZK,2019-09-30,8.0,9,2015-11-07,2020-11-06,35.919444,35.995833,22.125833,22.579444


In [74]:
# Subset the dataset based on lat/lon bounds, which is performed server-side
ds_conus = ds.sel(
    lat=slice(35,36), 
    lon=slice(22,23), 
    time=ds.time.dt.hour == 8
)
ds_conus.to_netcdf('merra_data/test.nc4')

# MERRA2 - pipeline

Input

In [57]:
os.getcwd()

'/Users/hakimdiouane/Documents/desert_locust_breeding_detection/benchmark_desert_locust_detection/Desert_Locust_Breeding_Detection/instageo_wxc/data_pipeline'

In [39]:
pd.read_csv('../latest_git_repo/geo_ai.hack/hls_data_sample/train.csv')

,Unnamed: 0,Input,Label
0,0,train/chips/chip_20200601_S30_T38PRC_2020125T0...,train/seg_maps/seg_map_20200601_S30_T38PRC_202...
1,1,train/chips/chip_20200301_S30_T40QFL_2020031T0...,train/seg_maps/seg_map_20200301_S30_T40QFL_202...
2,2,train/chips/chip_20200601_L30_T39PTM_2020121T0...,train/seg_maps/seg_map_20200601_L30_T39PTM_202...
3,3,train/chips/chip_20210101_L30_T37QGB_2020339T0...,train/seg_maps/seg_map_20210101_L30_T37QGB_202...
4,4,train/chips/chip_20200801_S30_T38KPE_2020184T0...,train/seg_maps/seg_map_20200801_S30_T38KPE_202...
5,5,train/chips/chip_20200101_S30_T40QGK_2019338T0...,train/seg_maps/seg_map_20200101_S30_T40QGK_201...
6,6,train/chips/chip_20200501_S30_T40REP_2020091T0...,train/seg_maps/seg_map_20200501_S30_T40REP_202...
7,7,train/chips/chip_20210401_S30_T32QMH_2021062T0...,train/seg_maps/seg_map_20210401_S30_T32QMH_202...
8,8,train/chips/chip_20200301_S30_T39RWM_2020030T0...,train/seg_maps/seg_map_20200301_S30_T39RWM_202...
9,9,train/chips/chip_20200601_S30_T35PPN_2020124T0...,train/seg_maps/seg_map_20200601_S30_T35PPN_202...


In [ ]:
from pydantic_settings import BaseSettings

class NoDataValues(BaseSettings):
    """Settings for no-data values to use for HLS, S2 and segmentation maps."""

    HLS: int = -9999
    S2: int = 0
    SEG_MAP: int = -1

In [ ]:
import logging
import os
from functools import partial
from typing import Any, Callable

import geopandas as gpd
import mgrs
import numpy as np
import pandas as pd
import planetary_computer
import xarray as xr
from pyproj import Transformer
from pystac_client import Client
from shapely.geometry import box

# Masks decoding positions
MASK_DECODING_POS: dict[str, dict] = {
    "HLS": {"cloud": 1, "water": 5},
    "S2": {"cloud": [8, 9], "water": [6]},
}

# No data values
NO_DATA_VALUES = NoDataValues().model_dump()


def create_and_save_chips_with_seg_maps(
    data_reader: Callable | partial,
    mask_fn: Callable,
    processing_method: str,
    tile_dict: dict[str, Any],
    data_source: str,
    df: pd.DataFrame,
    chip_size: int,
    output_directory: str,
    no_data_value: int,
    src_crs: int,
    mask_decoder: Callable,
    mask_types: list[str],
    masking_strategy: str,
    window_size: int,
) -> tuple[list[str], list[str | None]]:
    """Chip Creator.

    Create chips and corresponding segmentation maps from a satellite image tile and save
    them to an output directory.

    Args:
        data_reader (callable[dict[str, Any], bool] | functools.partial): A multi-file reader that
            accepts a dictionary of satellite image tile paths and reads it into an Xarray dataset
            or dataarray. Optionally performs masking based on the boolean mask types provided.
        mask_fn (Callable): Function to use to apply masks.
        processing_method (str): Processing method to use to create the chips and
        segmentation maps.
        tile_dict (Dict): A dict mapping band names to tile filepath.
        data_source (str): Data source, which can be "HLS", "S2" or "S1".
        df (pd.DataFrame): DataFrame containing the data for segmentation maps.
        chip_size (int): Size of each chip.
        output_directory (str): Directory where the chips and segmentation maps will be
            saved.
        no_data_value (int): Value to use for no data areas in the segmentation maps.
        src_crs (int): CRS of points in `df`
        mask_types (list[str]): Types of masking to perform.
        mask_decoder (Callable): Function to use to process/extract actual mask values
        masking_strategy (str): Masking strategy to apply ("each" for timestep-wise masking,
        and "any" to exclude pixels if the mask is present for at least one timestep. The
        behavior is the same if the chip is extracted for one timestep.)
        window_size (int): Window size to use around the observation pixel.

    Returns:
        A tuple containing the lists of created chips and segmentation maps.
    """
    load_masks = True if mask_types else False
    dsb, dsm, crs = data_reader(tile_dict, load_masks=load_masks)
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df.x, y=df.y))
    df.set_crs(epsg=src_crs, inplace=True)
    df = df.to_crs(crs=crs)
    df = df[
        (dsb["x"].min().item() <= df["geometry"].x)
        & (df["geometry"].x <= dsb["x"].max().item())
        & (dsb["y"].min().item() <= df["geometry"].y)
        & (df["geometry"].y <= dsb["y"].max().item())
    ]
    os.makedirs(output_directory, exist_ok=True)
    # TODO: handle chip names more gracefully
    tile_name_splits = (
        tile_dict["tiles"]["B02_0"].split(".")
        if data_source == "HLS"
        else tile_dict["granules"][0].split(".")[0].split("/")[-1].split("_")
    )
    tile_id = (
        f"{tile_name_splits[1]}_{tile_name_splits[2]}_{tile_name_splits[3]}"
        if data_source == "HLS"
        else f"{tile_name_splits[0]}_{tile_name_splits[1]}_"
        f"{tile_name_splits[5]}_{tile_name_splits[2]}"
    )
    date_id = df.iloc[0]["date"].strftime("%Y%m%d")
    chips = []
    seg_maps: list[str | None] = []

    n_chips_x = dsb.sizes["x"] // chip_size
    n_chips_y = dsb.sizes["y"] // chip_size
    chip_coords = get_chip_coords(df, dsb, chip_size)
    for x, y in chip_coords:
        # TODO: handle potential partially out of bound chips
        if (x >= n_chips_x) or (y >= n_chips_y):
            continue
        chip_id = f"{date_id}_{tile_id}_{x}_{y}"
        chip_name = f"chip_{chip_id}.tif"
        seg_map_name = f"seg_map_{chip_id}.tif"

        chip_filename = os.path.join(output_directory, "chips", chip_name)
        seg_map_filename = os.path.join(output_directory, "seg_maps", seg_map_name)
        if os.path.exists(chip_filename) or os.path.exists(seg_map_filename):
            continue
        chip = dsb.isel(
            x=slice(x * chip_size, (x + 1) * chip_size),
            y=slice(y * chip_size, (y + 1) * chip_size),
        ).compute()
        chip = chip if processing_method == "cog" else chip.band_data
        if dsm is not None:
            chip_mask = dsm.isel(
                x=slice(x * chip_size, (x + 1) * chip_size),
                y=slice(y * chip_size, (y + 1) * chip_size),
            ).compute()
            chip_mask = chip_mask if processing_method == "cog" else chip_mask.band_data
            chip = mask_fn(
                chip=chip,
                mask=chip_mask,
                no_data_value=no_data_value,
                mask_decoder=mask_decoder,
                data_source=data_source,
                mask_types=mask_types,
                masking_strategy=masking_strategy,
            )
        if chip.where(chip != no_data_value).count().values == 0:
            continue
        seg_map = create_segmentation_map(chip, df, window_size)
        if seg_map.where(seg_map != no_data_value).count().values == 0:
            continue
        seg_maps.append(seg_map_name)
        seg_map.rio.to_raster(seg_map_filename)
        chips.append(chip_name)
        chip.rio.to_raster(chip_filename)
    return chips, seg_maps


def apply_mask(
    chip: xr.DataArray,
    mask: xr.DataArray,
    no_data_value: int,
    mask_decoder: Callable,
    data_source: str,
    masking_strategy: str = "each",
    mask_types: list[str] = list(MASK_DECODING_POS["HLS"].keys()),
) -> xr.DataArray:
    """Apply masking to a chip.

    Args:
        chip (xr.DataArray): Chip array containing the pixels to be masked out.
        mask (xr.DataArray): Array containing the masks.
        no_data_value (int): Value to be used for masked pixels.
        mask_decoder (Callable): Function to use to process/extract actual mask values
        data_source (str): Data source used to extract masking positions based on mask types
        masking_strategy (str): Masking strategy to apply ("each" for timestep-wise masking,
        and "any" to exclude pixels if the mask is present for at least one timestep. The
        behavior is the same if the chip is extracted for one timestep.)
        mask_types (list[str]): Mask types to apply.

    Returns:
        xr.DataArray: The masked data array.
    """
    for mask_type in mask_types:
        pos = MASK_DECODING_POS[data_source].get(mask_type, None)
        if pos:
            decoded_mask = mask_decoder(mask, pos)
            if masking_strategy == "each":
                # repeat across timesteps so that, each mask is applied to its
                # corresponding timestep
                decoded_mask = decoded_mask.values.repeat(
                    chip.shape[0] // mask.shape[0], axis=0
                )
            elif masking_strategy == "any":
                # collapse the mask to exclude a pixel if its corresponding mask value
                # for at least one timestep is 1
                decoded_mask = decoded_mask.values.any(axis=0)
            chip = chip.where(decoded_mask == 0, other=no_data_value)
    return chip


def get_tile_info(
    data: pd.DataFrame,
    num_steps: int = 3,
    temporal_step: int = 10,
    temporal_tolerance: int = 5,
) -> tuple[pd.DataFrame, list[tuple[str, list[str]]]]:
    """Get Tile Info.

    Retrieves a summary of all tiles required for a given dataset. The summary contains
    the desired start and end date for each tile. Also retrieves a list of queries
    that can be used to retrieve the tiles for each observation in `data`.

    Args:
        data (pd.DataFrame): A dataframe containing observation records.
        num_steps (int): Number of temporal time steps
        temporal_step (int): Size of each temporal step.
        temporal_tolerance (int): Number of days used as offset for the
        start and end dates to search for each tile.

    Returns:
        A `tile_info` dataframe and a list of `tile_queries`
    """
    data = data[["mgrs_tile_id", "input_features_date", "x", "y"]].reset_index(
        drop=True
    )
    tile_queries = []
    tile_info: Any = []
    for _, (tile_id, date, lon, lat) in data.iterrows():
        history = []
        for i in range(num_steps):
            curr_date = date - pd.Timedelta(days=temporal_step * i)
            history.append(curr_date.strftime("%Y-%m-%d"))
            tile_info.append([tile_id, curr_date, lon, lat])
        tile_queries.append((tile_id, history))
    tile_info = (
        pd.DataFrame(tile_info, columns=["tile_id", "date", "lon", "lat"])
        .groupby("tile_id")
        .agg(
            min_date=("date", "min"),
            max_date=("date", "max"),
            lon_min=("lon", "min"),
            lon_max=("lon", "max"),
            lat_min=("lat", "min"),
            lat_max=("lat", "max"),
        )
    ).reset_index()
    tile_info["min_date"] -= pd.Timedelta(days=temporal_tolerance)
    tile_info["max_date"] += pd.Timedelta(days=temporal_tolerance)
    tile_info["min_date"] = tile_info["min_date"].dt.strftime("%Y-%m-%d")
    tile_info["max_date"] = tile_info["max_date"].dt.strftime("%Y-%m-%d")
    return tile_info, tile_queries


def reproject_coordinates(df: pd.DataFrame, source_epsg: int = 4326) -> pd.DataFrame:
    """Reproject coordinates from the source EPSG to EPSG:4326.

    This function reprojects the geo coordinates found in df dataframe to the EPSG:4326

    Args:
        df (pd.DataFrame): DataFrame containing longitude and latitude columns.
        source_epsg (int): The EPSG code of the source CRS for invalid coordinates.

    Returns:
        pd.DataFrame: DataFrame with transformed and valid coordinates.
    """
    logging.info("Reprojecting coordinates to EPSG:4326...")
    transformer = Transformer.from_crs(
        f"EPSG:{source_epsg}", "EPSG:4326", always_xy=True
    )

    # Reproject the invalid rows
    df[["x", "y"]] = df.apply(
        lambda row: transformer.transform(row["x"], row["y"]), axis=1
    )

    return df


def get_tiles(
    data: pd.DataFrame, src_crs: int = 4326, min_count: int = 100
) -> pd.DataFrame:
    """Retrieve Tile IDs for Geospatial Observations from Satellite Data.

    This function associates each geospatial observation with a tile ID based on its
    geographic location, accommodating datasets with varying density across locations. By
    focusing on more densely populated areas, it enables more efficient resource usage and
    refined data analysis.

    The function assigns a tile ID to each observation, counts the occurrences within
    each tile, and retains only those tiles with a specified minimum count (`min_count`) of
    observations.

    Args:
        data: DataFrame containing geospatial observations with location coordinates.
        src_crs (int): CRS of points in `data`
        min_count: Minimum count of observations required per tile to retain.

    Returns:
        A subset of observations within tiles that meet or exceed the specified `min_count`.
    """
    if src_crs != 4326:
        data = reproject_coordinates(data, source_epsg=src_crs)
    mgrs_object = mgrs.MGRS()
    get_mgrs_tile_id = lambda row: mgrs_object.toMGRS(
        row["y"], row["x"], MGRSPrecision=0
    )
    data["mgrs_tile_id"] = data.apply(get_mgrs_tile_id, axis=1)
    tile_counts = data.groupby("mgrs_tile_id").size().sort_values(ascending=False)
    data = pd.merge(
        data, tile_counts.reset_index(name="counts"), how="left", on="mgrs_tile_id"
    )
    sub_data = data[data["counts"] >= min_count]
    assert not sub_data.empty, "No observation records left"
    return sub_data

In [45]:
data

,Unnamed: 0,Input,Label
0,0,train/chips/chip_20200601_S30_T38PRC_2020125T0...,train/seg_maps/seg_map_20200601_S30_T38PRC_202...
1,1,train/chips/chip_20200301_S30_T40QFL_2020031T0...,train/seg_maps/seg_map_20200301_S30_T40QFL_202...
2,2,train/chips/chip_20200601_L30_T39PTM_2020121T0...,train/seg_maps/seg_map_20200601_L30_T39PTM_202...
3,3,train/chips/chip_20210101_L30_T37QGB_2020339T0...,train/seg_maps/seg_map_20210101_L30_T37QGB_202...
4,4,train/chips/chip_20200801_S30_T38KPE_2020184T0...,train/seg_maps/seg_map_20200801_S30_T38KPE_202...
5,5,train/chips/chip_20200101_S30_T40QGK_2019338T0...,train/seg_maps/seg_map_20200101_S30_T40QGK_201...
6,6,train/chips/chip_20200501_S30_T40REP_2020091T0...,train/seg_maps/seg_map_20200501_S30_T40REP_202...
7,7,train/chips/chip_20210401_S30_T32QMH_2021062T0...,train/seg_maps/seg_map_20210401_S30_T32QMH_202...
8,8,train/chips/chip_20200301_S30_T39RWM_2020030T0...,train/seg_maps/seg_map_20200301_S30_T39RWM_202...
9,9,train/chips/chip_20200601_S30_T35PPN_2020124T0...,train/seg_maps/seg_map_20200601_S30_T35PPN_202...


In [47]:
data

,Unnamed: 0,x,y,date,label,generated,CAT,year,input_features_date,mgrs_tile_id
0,6507,38.688200,8.409860,2020-04-01,0.0,1,PseudoAbsence,2020,2020-03-02,37PDK
1,22,28.588200,11.109860,2020-06-01,0.0,1,PseudoAbsence,2020,2020-05-02,35PPN
2,1071,40.688200,30.509860,2020-10-01,0.0,1,PseudoAbsence,2020,2020-09-01,37RFP
3,4038,8.588200,20.409860,2021-04-01,0.0,1,PseudoAbsence,2021,2021-03-02,32QMH
4,5759,46.788200,-18.690140,2020-08-01,0.0,1,PseudoAbsence,2020,2020-07-02,38KPE
5,2503,1.488200,8.109860,2020-02-01,0.0,1,PseudoAbsence,2020,2020-01-02,31PCJ
6,4664,72.688200,22.309860,2020-06-01,0.0,1,PseudoAbsence,2020,2020-05-02,43QBE
7,241,54.588200,24.509860,2020-01-01,0.0,1,PseudoAbsence,2020,2019-12-02,40RBN
8,588,0.088200,20.509860,2020-07-01,0.0,1,PseudoAbsence,2020,2020-06-01,31QAC
9,7519,24.288200,21.809860,2020-10-01,0.0,1,PseudoAbsence,2020,2020-09-01,35QKE


In [51]:
pd.read_csv('hls_data_sample/records/train.csv')

,Unnamed: 0,x,y,date,label,generated,CAT,year
0,6507,38.688200,8.409860,2020-04-17,0.0,1,PseudoAbsence,2020
1,22,28.588200,11.109860,2020-06-23,0.0,1,PseudoAbsence,2020
2,1071,40.688200,30.509860,2020-10-08,0.0,1,PseudoAbsence,2020
3,4038,8.588200,20.409860,2021-04-16,0.0,1,PseudoAbsence,2021
4,5759,46.788200,-18.690140,2020-08-31,0.0,1,PseudoAbsence,2020
5,2503,1.488200,8.109860,2020-02-19,0.0,1,PseudoAbsence,2020
6,4664,72.688200,22.309860,2020-06-11,0.0,1,PseudoAbsence,2020
7,241,54.588200,24.509860,2020-01-31,0.0,1,PseudoAbsence,2020
8,588,0.088200,20.509860,2020-07-18,0.0,1,PseudoAbsence,2020
9,7519,24.288200,21.809860,2020-10-04,0.0,1,PseudoAbsence,2020


In [50]:
data = pd.read_csv('hls_data_sample/records/train.csv')
data["date"] = (
    pd.to_datetime(data["date"]) - pd.offsets.MonthBegin(1)
    if True #FLAGS.shift_to_month_start
    else pd.to_datetime(data["date"])
)
data["input_features_date"] = (
    data["date"] - pd.DateOffset(days= 30)#FLAGS.temporal_step)
    if True #FLAGS.is_time_series_task
    else data["date"]
)
num_steps = 1 if not True else 3 #FLAGS.is_time_series_task else FLAGS.num_steps
sub_data = get_tiles(data, src_crs=4326, min_count=1)#FLAGS.src_crs, min_count=FLAGS.min_count)

sub_data

,Unnamed: 0,x,y,date,label,generated,CAT,year,input_features_date,mgrs_tile_id,counts
0,6507,38.688200,8.409860,2020-04-01,0.0,1,PseudoAbsence,2020,2020-03-02,37PDK,1
1,22,28.588200,11.109860,2020-06-01,0.0,1,PseudoAbsence,2020,2020-05-02,35PPN,1
2,1071,40.688200,30.509860,2020-10-01,0.0,1,PseudoAbsence,2020,2020-09-01,37RFP,1
3,4038,8.588200,20.409860,2021-04-01,0.0,1,PseudoAbsence,2021,2021-03-02,32QMH,1
4,5759,46.788200,-18.690140,2020-08-01,0.0,1,PseudoAbsence,2020,2020-07-02,38KPE,1
5,2503,1.488200,8.109860,2020-02-01,0.0,1,PseudoAbsence,2020,2020-01-02,31PCJ,1
6,4664,72.688200,22.309860,2020-06-01,0.0,1,PseudoAbsence,2020,2020-05-02,43QBE,1
7,241,54.588200,24.509860,2020-01-01,0.0,1,PseudoAbsence,2020,2019-12-02,40RBN,1
8,588,0.088200,20.509860,2020-07-01,0.0,1,PseudoAbsence,2020,2020-06-01,31QAC,1
9,7519,24.288200,21.809860,2020-10-01,0.0,1,PseudoAbsence,2020,2020-09-01,35QKE,1
